In [22]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem

In [23]:
p_1, p_2, p_3 = sympy.symbols("p_1 p_2 p_3")
p_1_x, p_2_x, p_1_y, p_2_y = sympy.symbols("p_1_x p_2_x p_1_y p_2_y")
right_vals = sympy.symbols("f_1:7")
x, y = sympy.symbols("x, y")

In [24]:
all_basis = symfem.create_element("triangle", "Lagrange", 2).get_basis_functions()
all_basis = [itm.subs(x, p_1).subs(y, p_2) for itm in all_basis]

In [25]:
N = len(all_basis)

In [26]:
result = {}
args = [p_1_x, p_2_x, p_1_y, p_2_y]

J_inv_det = sympy.Matrix(args).reshape(2, 2).inv().det()

result_A = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]


for i in tqdm(range(6)):
    for j in range(6):
        first = all_basis[i].subs(p_3, 1 - p_1 - p_2)
        second = all_basis[j].subs(p_3, 1 - p_1 - p_2)

        u_x = first.diff(p_1) * p_1_x + first.diff(p_2) * p_2_x
        u_y = first.diff(p_1) * p_1_y + first.diff(p_2) * p_2_y

        v_x = second.diff(p_1) * p_1_x + second.diff(p_2) * p_2_x
        v_y = second.diff(p_1) * p_1_y + second.diff(p_2) * p_2_y

        integral_A = (
            sympy.integrate(u_x * v_x + u_y * v_y, (p_2, 0, 1 - p_1), (p_1, 0, 1))
            * J_inv_det
        )
        integral_B = (
            sympy.integrate(first * second, (p_2, 0, 1 - p_1), (p_1, 0, 1)) * J_inv_det
        )

        result_A[i][j] = integral_A
        result_B[i][j] = integral_B

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.47s/it]


In [29]:
result_A = sympy.Matrix(result_A)
result_B = sympy.Matrix(result_B)
right_vals = sympy.Matrix(right_vals)

In [30]:
syms = [*args, *right_vals]

In [31]:
FUNC_A = sympy.lambdify(syms, result_A)

In [20]:
FUNC_B = sympy.lambdify(syms, result_B @ right_vals)

In [32]:
import dill

dill.settings["recurse"] = True
dill.dump(FUNC_A, open("../calculations/lagrange_quadratic_poisson_matrix", "wb"))
dill.dump(FUNC_B, open("../calculations/lagrange_quadratic_poisson_right_values", "wb"))